In [1]:
import numpy as np
import tqdm 
import trimesh

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    '''
    vertices: n个顶点
    faces: n*3，3个点的序号
    '''
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    
    # 1. compute area of each triangles
    # 找出三角形的三个点
    X = vertices[faces[:, 0]]
    Y = vertices[faces[:, 1]]
    Z = vertices[faces[:, 2]]
    # 转换为三边
    A = np.apply_along_axis(np.linalg.norm, -1, X-Y)
    B = np.apply_along_axis(np.linalg.norm, -1, Y-Z)
    C = np.apply_along_axis(np.linalg.norm, -1, Z-X)
    # 海伦公式计算面积
    P = (A + B + C) / 2
    area = np.sqrt(P * (P-A) * (P-B) * (P-C))
    # 2. compute probability of each triangles from areas
    prob = area / np.sum(area)
    # 3. sample N faces according to the probability
    sampled_faces_idx = np.random.choice(np.arange(faces.shape[0]), size=sample_num, p=prob)
    # 4. for each face, sample 1 point
    sampled_X = vertices[faces[sampled_faces_idx, 0]]
    sampled_Y = vertices[faces[sampled_faces_idx, 1]]
    sampled_Z = vertices[faces[sampled_faces_idx, 2]]
    array1 = sampled_Y - sampled_X
    array2 = sampled_Z - sampled_X
    weight1 = np.random.rand(array1.shape[0]) / 2
    weight2 = np.random.rand(array2.shape[0]) / 2
    uniform_pc = sampled_X + weight1.reshape((-1,1)) * array1 + weight2.reshape((-1,1)) * array2
    return area, prob, uniform_pc


In [3]:
from tqdm import trange

def farthest_point_sampling(pc, sample_num):
    # FOR LOOP is allowed here.
    
    PC1 = pc.reshape((pc.shape[0], 1, 3)) # (N,1,3)
    PC2 = pc.reshape((1, pc.shape[0], 3))
    dist = np.apply_along_axis(np.linalg.norm, -1, PC1-PC2) # (N,N)
    # 1. 找一个起始点，加入初始点集S
    S = np.random.choice(pc.shape[0],1)
    # 2. 找到距离S最远的点，将其加入S，直到达到采样个数
    # 这里每个循环中都算一遍距离太慢了，应该先计算所有点之间的距离，避免重复计算。
    for i in range(sample_num-1):
        dist_s = np.sum(dist[:,S], axis=-1)
        idx = np.argmax(dist_s)
        S = np.append(S, idx)
    
    results = pc[S]
    return results


In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)
'''
OUTPUT:
faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)
'''

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


'\nOUTPUT:\nfaces shape:  (13712, 3)\narea shape:  (13712,)\nprob shape:  (13712,)\npc shape:  (512, 3)\n'

In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [6]:
# task 3: metrics
# 直接把 earthmover.py 放在
from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
import time
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------
# uniform_pc, tmp_pc
print(uniform_pc.shape, tmp_pc.shape)
def chamfer_distance(u_pc, t_pc):
    U = np.tile(u_pc[:,np.newaxis,:], (1,t_pc.shape[0],1))
    T = np.tile(t_pc[np.newaxis,:,:], (u_pc.shape[0],1,1))
    dist = np.apply_along_axis(np.linalg.norm, 2, U-T) #(512,2000)
    cd = np.sum(np.min(dist, axis=1)) / dist.shape[0] + np.sum(np.min(dist, axis=0)) / dist.shape[1]
    return cd

N = 5
cd = np.zeros(N)
emd = np.zeros(N)
for i in trange(N):
    t0 = time.time()
    print('Do uniform sampling...')
    area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    print('Do farthest point sampling...')
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)
    print('Compute CD...')
    cd[i] = chamfer_distance(uniform_pc, tmp_pc)
    print('Compute EMD...')
    emd[i] = earthmover_distance(uniform_pc, tmp_pc)
    t1 = time.time()
    print(f'Result: CD {cd[i]}, EMD {emd[i]}')
    print(f'Iteration {i} cost {t1-t0} seconds.')


CD_mean = np.mean(cd)
CD_var = np.var(cd)
EMD_mean = np.mean(emd)
EMD_var = np.mean(emd)

print('Report:', CD_mean, CD_var, EMD_mean, EMD_var)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

(512, 3) (2000, 3)


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Do uniform sampling...
Do farthest point sampling...
Compute CD...
Compute EMD...


 20%|████████████████▌                                                                  | 1/5 [04:09<16:38, 249.60s/it]

Result: CD 2.317574241495713, EMD 2.241970597874175
Iteration 0 cost 249.59579300880432 seconds.
Do uniform sampling...
Do farthest point sampling...
Compute CD...
Compute EMD...


...

Result: CD 2.3381654686281603, EMD 2.382991782154591
Iteration 2 cost 249.48699498176575 seconds.
Do uniform sampling...
Do farthest point sampling...
Compute CD...
Compute EMD...


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [16:43<04:10, 250.28s/it]

Result: CD 2.2631610777225193, EMD 2.2324813210372936
Iteration 3 cost 247.98847818374634 seconds.
Do uniform sampling...
Do farthest point sampling...
Compute CD...
Compute EMD...


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [20:49<00:00, 249.87s/it]

Result: CD 2.2025803242848503, EMD 2.3048623053403574
Iteration 4 cost 245.72123885154724 seconds.
Report: 2.277198835137972 0.002254602686687156 2.2698422716797184 2.2698422716797184
